<a href="https://colab.research.google.com/github/maddieemihle/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Sales Analysis

### Step 1: Import Libraries

In [3]:
import os

# Set Spark version
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [8]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import pandas as pd
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

### Step 2: Read the Data

In [10]:
# Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
url_df = pd.read_csv(url)


In [11]:
# Create a temporary view of the DataFrame.
df = spark.createDataFrame(url_df)
df.createOrReplaceTempView("home_sales")
df.show(5)


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

### Step 3: Analyze Using SparkSQL

In [13]:
# Query 1: What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query_1 = """
SELECT year(date) AS year_sold, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year_sold
ORDER BY year_sold
"""

# Execute the query and show the results.
result_1 = spark.sql(query_1)
result_1.show()

+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [16]:
# Query 2: What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query_2 = """
SELECT date_built AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query and show the results
result_2 = spark.sql(query_2)
result_2.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [18]:
# Query 3: What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query_3 = """
SELECT date_built AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query and show the results.
result_3 = spark.sql(query_3)
result_3.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [19]:
# Query 4: What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query_4 = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""

# Execute the query and show the results.
result_4 = spark.sql(query_4)
result_4.show()

# Print the run time.
end_time = time.time()
print("Query 4 run time: %s seconds" % (end_time - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Query 4 run time: 1.3329832553863525 seconds


In [20]:
# Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")


DataFrame[]

In [21]:
# Check if the table is cached.
print("Is the table cached?", spark.catalog.isCached("home_sales"))

Is the table cached? True


In [22]:
# Run Query 4 again using the cached table and measure runtime
start_time = time.time()
result_4_cached = spark.sql(query_4)
result_4_cached.show()

# Print the run time.
end_time = time.time()
print("Query 4 cached run time: %s seconds" % (end_time - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Query 4 cached run time: 0.7386057376861572 seconds


In [23]:
# Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("home_sales_partitioned.parquet")

# Read the formatted parquet data.
parquet_df = spark.read.parquet("home_sales_partitioned.parquet")
parquet_df.show()

+--------------------+----------+-------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+-------+--------+---------+-----------+--------+------+----------+----+----------+
|62c69180-ab61-47c...|2020-07-06| 395216|       4|        2|       2428|    8908|     2|         0|  20|      2017|
|273b36f4-26eb-4dd...|2021-04-27| 306223|       2|        2|       2181|   12568|     1|         0|   2|      2017|
|e2919f3b-af95-464...|2021-12-12| 152105|       2|        3|       1998|   11236|     2|         0|  21|      2017|
|1c0eeb36-0f40-479...|2022-04-07| 150817|       3|        3|       2283|    8650|     2|         0|   8|      2017|
|29a05191-0b08-4ac...|2021-03-02|1471564|       8|        3|       5088|   10228|     2|         0|  97|      2017|
|db7e9f31-e4fd-407...|2019-12-29| 152809|       3|        3|       1834|

In [24]:
# Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [25]:
# Run Query 4 on the Parquet data and measure runtime
start_time = time.time()
result_4_parquet = spark.sql(query_4)
result_4_parquet.show()

# Print the run time.
end_time = time.time()
print("Query 4 parquet run time: %s seconds" % (end_time - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Query 4 parquet run time: 0.8703858852386475 seconds


In [26]:
# Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")


DataFrame[]

In [27]:
# Check if the home_sales is no longer cached
print("Is the table cached?", spark.catalog.isCached("home_sales"))


Is the table cached? False


Resources: Data for this dataset was generated by _edX Boot Camps LLC_, and is intended for educational purposes only.